<a href="https://colab.research.google.com/github/croco22/CapstoneProjectTDS/blob/philipp/notebooks/ExtraTask2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extra Task 2: Continuous Text Evaluation

## Imports and Setup

In [1]:
import json
import time
import re
from google.colab import userdata
import google.generativeai as genai
import pandas as pd
from transformers import pipeline
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [2]:
# Read dataset files
url = 'https://raw.githubusercontent.com/croco22/CapstoneProjectTDS/refs/heads/philipp/qa_dataset_continuous.json' # Todo: Change branch
continuous_data = pd.read_json(url)

url2 = 'https://raw.githubusercontent.com/croco22/CapstoneProjectTDS/refs/heads/main/qa_dataset.json'
data = pd.read_json(url2)

continuous_data.head()

,text,questions
0,"In regards to the customer group, it's importa...","[{'question': 'Customer group', 'intended_answ..."
1,"Nope, definitely not, please don’t send me any...",[{'question': 'Would you like to receive marke...
2,"And finally, and this is something I always ap...",[{'question': 'How could this app better suppo...
3,"Also, sometimes I think it’s just a tad slow t...",[{'question': 'How could we improve your exper...
4,"Finally, something I'm struggling with quite a...",[{'question': 'What specific challenges are yo...


In [3]:
model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
for text in continuous_data['text']:
    sentences = [sentence.strip(' "') for sentence in re.split(r'[.!?]', text) if sentence]
    sentence_embeddings = model.encode(sentences)
    similarity_matrix = cosine_similarity(sentence_embeddings)
    clustering = AgglomerativeClustering(n_clusters=3, metric='cosine', linkage='average')
    labels = clustering.fit_predict(sentence_embeddings)

    clusters = {}
    for sentence, label in zip(sentences, labels):
        clusters.setdefault(label, []).append(sentence)

    for cluster_id, grouped_sentences in clusters.items():
        print(f"Cluster {cluster_id}:")
        for sentence in grouped_sentences:
            print(f"  - {sentence}")
    print("\n\n")

Cluster 2:
  - In regards to the customer group, it's important to note that we are talking about R&D, that is, the research and development section of the organisation
Cluster 1:
  - Let me see, my best guess is that it'll be done in approximately three months, that's the current timeline
Cluster 0:
  - My job duties aren't straightforward, and I tend to wear many hats, so if I had to label myself, I suppose I would say my primary role in the organization is Other



Cluster 2:
  - Nope, definitely not, please don’t send me any promotional emails, I’d really appreciate it
Cluster 1:
  - Yes, I can do that, no problem at all, my phone number is +1-147-649-9205, I look forward to hearing from you
Cluster 0:
  - Okay, so after looking over the choices, and thinking about it a little bit, I'd have to say that my primary goal falls under the category of 'Other', since none of the given options are quite right for me



Cluster 0:
  - And finally, and this is something I always appreciate i